In [2]:
from ultralytics import YOLO
import pandas as pd
import onnxruntime as ort
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics

from pathbook.pathbook import *

In [13]:
model = YOLO('../bin/yolov8x-seg.onnx')
results = []
results += model.predict(path_val_dataset+'/klikun',imgsz=320,batch=100)
results += model.predict(path_val_dataset+'/maliy',imgsz=320,batch=100)
results += model.predict(path_val_dataset+'/shipun',imgsz=320,batch=100)
df = pd.read_csv(path_val_annotation)

Loading ../bin/yolov8x-seg.onnx for ONNX Runtime inference...

image 1/452 /Users/samedi/Documents/Coding/swans/dataset/val/klikun/initial-klikun-1011.jpg: 320x320 1 klikun, 566.0ms
image 2/452 /Users/samedi/Documents/Coding/swans/dataset/val/klikun/initial-klikun-1012.jpg: 320x320 1 klikun, 858.6ms
image 3/452 /Users/samedi/Documents/Coding/swans/dataset/val/klikun/initial-klikun-1013.jpg: 320x320 2 klikuns, 797.0ms
image 4/452 /Users/samedi/Documents/Coding/swans/dataset/val/klikun/initial-klikun-1024.jpg: 320x320 2 klikuns, 676.8ms
image 5/452 /Users/samedi/Documents/Coding/swans/dataset/val/klikun/initial-klikun-1029.jpg: 320x320 2 klikuns, 761.9ms
image 6/452 /Users/samedi/Documents/Coding/swans/dataset/val/klikun/initial-klikun-1045.jpg: 320x320 1 klikun, 782.3ms
image 7/452 /Users/samedi/Documents/Coding/swans/dataset/val/klikun/initial-klikun-1047.jpg: 320x320 2 klikuns, 851.5ms
image 8/452 /Users/samedi/Documents/Coding/swans/dataset/val/klikun/initial-klikun-1048.jpg: 320x320

In [14]:
y_pred = []
y_true = []

for result in results:
    sp = result.path.split('/')
    answer_df = df[df.path == sp[-2]+'/'+sp[-1]]['class_name']
    y_true.append(answer_df.iloc[answer_df.count().argmax()])

    bboxes = pd.DataFrame(result.boxes.data, columns=['x1','y1','x2','y2','conf','class'])
    if bboxes.size == 0:
        y_pred.append('maliy')
        continue
    y_pred.append(result.names[bboxes.iloc[bboxes['conf'].idxmax()]['class']])

print(metrics.accuracy_score(y_true, y_pred))

0.9793205317577548


|           | size | time | acc submit |acc val|
|---|---|---|---|---|
|yolov7      |  640 |  270 |   0.9126 | 0.9845|
|yolov8n-seg |  320 |   20 |   0.8907 | 0.9490|
|yolov8s-seg | 320 |   47 |   0.9210 | 0.9704|
|yolov8s-seg  | 640 |  174 |   0.9294 | 0.9830|
|yolov8x-seg  | 320 |  386 |   0.9193 | 0.9793|
|---|---|---|---|---|
|complex-o | 320 | ~200 | 0.9003 | 0.9747|